## Imports


In [2]:
import numpy as np
from icecream import ic
from gxgp import  Node
from tqdm import tqdm
import operator
import inspect

In [2]:
def true_f(x: np.ndarray) -> np.ndarray:
    return x[0] + np.sin(x[1]) / 5

In [14]:
def my_f(x: np.ndarray) -> np.ndarray:
    return np.add(np.minimum(np.log(54.42295941438421),np.maximum(x[0],x[0])),np.divide(x[1],57.335741198629776))

In [20]:
TEST_SIZE = 10_000
TRAIN_SIZE = 1000

def gen_problem():
    x_validation = np.vstack(
        [
            np.random.random_sample(size=TEST_SIZE) * 2 * np.pi - np.pi,
            np.random.random_sample(size=TEST_SIZE) * 2 - 1,
        ]
    )
    y_validation = true_f(x_validation)
    train_indexes = np.random.choice(TEST_SIZE, size=TRAIN_SIZE, replace=False)
    x_train = x_validation[:, train_indexes]
    y_train = y_validation[train_indexes]
    assert np.all(y_train == true_f(x_train)), "D'ho"
    return x_train, y_train

# np.savez('problem_0.npz', x=x_train, y=y_train)
problem = np.load('problem_0.npz')
x = problem['x']
y = problem['y']
x.shape[0]
number_vars = x.shape[0]
# compute mse with truef as reference
mse = true_f(x) - my_f(x)
mse 



array([-0.07172609,  0.10578362, -0.09035033, -0.11625502, -0.07934737,
        0.14224806, -0.13665881, -0.04256511,  0.03939215,  0.0175185 ,
        0.07190767, -0.01633045, -0.04688148,  0.02875393, -0.071045  ,
       -0.14143438,  0.13349357,  0.13863918,  0.1111592 ,  0.12650503,
        0.06448448, -0.0433292 ,  0.14922759, -0.0985999 ,  0.0253658 ,
       -0.0968691 ,  0.12089605, -0.10760083, -0.1247588 ,  0.07871869,
        0.14493113,  0.00650947,  0.01841362, -0.08488751, -0.06044193,
        0.06869695,  0.08585437, -0.0907098 ,  0.04776554, -0.06136389,
        0.02374547, -0.05256879,  0.04053774, -0.07112301,  0.13456036,
       -0.01327813, -0.13571356,  0.09109564, -0.03961972, -0.00354262,
        0.09549216, -0.08748476,  0.10147455,  0.10557927, -0.08088743,
       -0.08516135,  0.14242501, -0.13781074,  0.04159387, -0.15074914,
        0.13677378, -0.12973162,  0.14015145, -0.1488284 , -0.05207867,
        0.00070597, -0.11302228,  0.04144739, -0.14975218, -0.11

### Getting numpy functions

In [ ]:
def get_np_functions()->list[tuple[np.ufunc,str,int]]:
    # Get all attributes of numpy module
    all_attrs = dir(np)

    # Filter to include only ufunc objects
    ufuncs = [getattr(np, attr) for attr in all_attrs if isinstance(getattr(np, attr), np.ufunc)]
    # Filter to include only common ufuncs
    # ufunc_list = [(ufunc,ufunc.__name__,ufunc.nin) for ufunc in ufuncs if ufunc.__name__ in ['absolute','add','subtract','multiply','divide','sin','cos','sinh','cosh','asin','asinh','acos','acosh','sqrt','exp','exp2','log2','log','maximum','minimum',]]
    ufunc_list = [ufunc for ufunc in ufuncs if ufunc.__name__ in ['absolute','add','subtract','multiply','divide','sin','cos','sinh','cosh','asin','asinh','acos','acosh','sqrt','exp','exp2','log2','log','maximum','minimum',]]
    return ufunc_list


np.float64(0.5508967046716147)

## Symbolic Regressor


### Utilify Functions


In [5]:
from tree3 import TweakableBTree
def RHH_Init():
    pass
def fitness(tree:TweakableBTree,x_train:np.ndarray,y_train:np.ndarray)->float:
    res = [tree.evaluate(x) for x in x_train.T]
    return np.mean((res-y_train)**2)
def fitness_proportional_selection(trees:list[TweakableBTree],n_elems=2)->list[TweakableBTree]:
    """
    Windowing version of fitness proportional selection
    Each tree has a probability of (fitness-min(fitnesses))/(sum(fitnesses)-min(fitnesses)) to be chosen
    """
    fitnesses = np.array([tree.fitness for tree in trees])
    # print(fitnesses)
    fitnesses = fitnesses - np.min(fitnesses)
    fitnesses = fitnesses / np.sum(fitnesses)
    # replace NaN with an high value
    return np.random.choice(trees, n_elems, p=fitnesses)
def tournament_selection(trees:list[TweakableBTree],n_elems=2)->list[TweakableBTree]:
    """
    Tournament selection
    """
    challengers = np.random.choice(trees, n_elems)
    return sorted(challengers, key=lambda x: x.fitness)[0]


def recombination(tree1:TweakableBTree,tree2:TweakableBTree)->tuple[TweakableBTree]:
    """
    Recombination of two trees
    Two parents swap their subtrees at a random node
    """
    t1copy = tree1.clone()
    t2copy = tree2.clone()
    # 1 - Select a random node from each tree
    t1node = t1copy.get_random_operator()
    t2node = t2copy.get_random_operator()
    # 2 - Swap the nodes
    tmp = t1node.value
    tmpl = t1node.left
    tmpr = t1node.right
    tmpt = t1node.type

    t1node.value = t2node.value
    t1node.left = t2node.left
    t1node.right = t2node.right
    t1node.type = t2node.type
    
    t2node.value = tmp
    t2node.left = tmpl
    t2node.right = tmpr
    t2node.type = tmpt
    return t1copy,t2copy
   
def mutation(tree:TweakableBTree,max_const:int,md:int)->TweakableBTree:
    """
    Mutation of a tree
    Randomly select a mutation method and apply it
    """
    mutation_kind = np.random.randint(5)
    match mutation_kind:
        case 0:
            tree.point_mutation(max_const=max_const)
            return tree
        case 1:
            tree.expansion_mutation(md)
            return tree
        case 2:
            tree.hoist_mutation()
            return tree
        case 3:
            tree.subtree_mutation(md)
            return tree
        case 4:
            tree.collapse_subtree_mutation()
            return tree




### Regression 

In [ ]:
N_VARS = number_vars
MD = 3 # Maximum depth
MAX_CONST = 10 # Maximum constant value
OFFSPRING_SIZE = 20
GENERATIONS = 100
PM = 0.05

operator_list = get_np_functions()
population = [TweakableBTree.generate_random_tree_growfull(operator_list,N_VARS,MD,True,PM) for _ in range(OFFSPRING_SIZE)]

for elem in population:
    elem.fitness = fitness(elem,x,y)
    if np.isnan(elem.fitness) or np.isinf(elem.fitness):
        elem.fitness = np.float64(1000)
    # ic(elem.fitness)
offspring = []
for generation in tqdm(range(GENERATIONS)):
    # parent selection
    # parents = fitness_proportional_selection(population)
    parents = [tournament_selection(population) for _ in range(2)]

    # recombination
    for _ in range(OFFSPRING_SIZE):
        o1,o2 = recombination(parents[0],parents[1])
        offspring.append(o1)
        offspring.append(o2)

    if np.random.rand() < PM:
        offspring_idx = np.random.randint(OFFSPRING_SIZE)
        mutation(population[offspring_idx],MAX_CONST,MD)

    for elem in offspring:
        elem.fitness = fitness(elem,x,y)
        if np.isnan(elem.fitness)  or np.isinf(elem.fitness):
            elem.fitness = np.float64(1000)
        # ic(elem.fitness)
    # ic("After recombination")

    # mutation
    

    # survivor selection
    population.extend(offspring)
    for elem in population:
        elem.fitness = fitness(elem,x,y)
    population.sort(key=operator.attrgetter('fitness'))
    population = population[:OFFSPRING_SIZE]

ic([elem for elem in population if not np.isnan(elem.fitness) and not  np.isinf(elem.fitness)])

    
    









c:\Users\denni\Desktop\PythonStuff\CompInt\Labs\CI2024_lab4\tree3.py:93: RuntimeWarning: invalid value encountered in log2
  tmp_res = self.value[0](left)
c:\Users\denni\Desktop\PythonStuff\CompInt\Labs\CI2024_lab4\tree3.py:93: RuntimeWarning: invalid value encountered in sqrt
  tmp_res = self.value[0](left)
c:\Users\denni\Desktop\PythonStuff\CompInt\Labs\CI2024_lab4\tree3.py:93: RuntimeWarning: invalid value encountered in log
  tmp_res = self.value[0](left)
c:\Users\denni\Desktop\PythonStuff\CompInt\Labs\CI2024_lab4\tree3.py:93: RuntimeWarning: overflow encountered in sinh
  tmp_res = self.value[0](left)
  3%|▎         | 3/100 [00:06<03:37,  2.25s/it]c:\Users\denni\Desktop\PythonStuff\CompInt\Labs\CI2024_lab4\tree3.py:86: RuntimeWarning: divide by zero encountered in divide
  tmp_res = self.value[0](left,right)
  6%|▌         | 6/100 [00:19<06:10,  3.94s/it]c:\Users\denni\Desktop\PythonStuff\CompInt\Labs\CI2024_lab4\tree3.py:93: RuntimeWarning: divide by zero encountered in log2
  tm

ValueError: high <= 0

In [ ]:
population[2].to_np_formula()
np.random.randint()

NameError: name 'idx' is not defined